Import libraries:

In [ ]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import ipywidgets as widgets
import os
import time
from IPython.display import display
from IPython.display import clear_output
from IPython.display import HTML
from IPython import get_ipython

imuData object:

In [2]:
class imuData:
    def __init__(self, name, df, sensor_num):
        self.name = name
        self.indices = [row for row in df.index] # list of analog labels

        self.start_row_index = self.indices.index(
            'DelsysTrignoBase 1: Sensor '+str(sensor_num)+'IM ACC Pitch')

        self.all_data = df.iloc[self.start_row_index : self.start_row_index+6]

        acc_data = self.all_data.iloc[0:3]
        self.a_local = acc_data.transpose()
        self.a_local.columns.values[0:3] = ['a_pitch', 'a_roll', 'a_yaw']

        sqrt_acc = np.square(self.a_local)
        net_acc_sq = sqrt_acc.apply(np.sum, axis=1, raw=True)
        self.net_acc = np.sqrt(net_acc_sq) # net acc for each frame

        gyr_data = self.all_data.iloc[3:7]
        self.omega_local = gyr_data.transpose()
        self.omega_local.columns.values[0:3] = ['omega_pitch', 'omega_roll', 'omega_yaw']

        self.measurements = self.a_local.join(self.omega_local)

        self.frames = len(acc_data.columns)

    def __str__(self):
        return f"ImuData object.\nname:  '{self.name}'\nframes: {self.frames}"

Kalman filter function:

In [ ]:
class KF6axIMU:
    def __init__(self, measurements, dt, Q, R, x_0, P_0):
        if measurements.shape[1] == 6:
            self.measurements = measurements
        else:
            print(f'"measurements" shape error: expected 6 columns, got {measurements.shape[1]} instead')
        self.dt = float(dt)
        if Q.shape == (16,16):
            self.Q = Q
        else:
            print(f'"Q" shape error: expected (16, 16) columns, got {Q.shape} instead')
        

        

In [10]:
Qtest = np.diag([1.]*16)
Qtest.shape == (16,16)

True

Participant dropdown menu:

In [5]:
# Get the list of subdirectories
participants = [d for d in os.listdir('trialData')]

# Create the participant dropdown widget with subdirectories as options
selected_participant = widgets.Dropdown(
    options=participants,
    value=participants[0] if participants else None,
    description='Participant:',
    disabled=False,
)

# Display the participant dropdown
display(selected_participant)

# Function to handle participant changes
def on_participant_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected_participant = change['new']
        # print(f'Selected participant: {selected_participant}')
        
        # Update the trial dropdown based on the selected participant
        trials = [d for d in os.listdir(f'trialData/{selected_participant}')]
        selected_trial.options = trials
        selected_trial.value = trials[0] if trials else None

# Attach the handler to the participant dropdown
selected_participant.observe(on_participant_change, names='value')

# Initial list of trials for the default participant
trials = [d for d in os.listdir(f'trialData/{selected_participant.value}')]

# Create the trial dropdown widget with subdirectories as options
selected_trial = widgets.Dropdown(
    options=trials,
    value=trials[0] if trials else None,
    description='Trial:',
    disabled=False,
)

output = widgets.Output()
# Display the trial dropdown
display(selected_trial)
display(output)

# Function to handle trial changes
def on_trial_change(change):
    global eg_measurements
    if change['type'] == 'change' and change['name'] == 'value':
        with output:
            clear_output(wait=True)
            display(HTML(f'<b>Getting measurements from "trialData/{selected_participant.value}/{change["new"]}":</b>'))
            # print('\n')
            trial_path = f'trialData/{selected_participant.value}/{change["new"]}'
            eg_df = pd.read_csv(trial_path, index_col=0)
            eg_imu = imuData('egIMU', eg_df, 4)
            eg_measurements = eg_imu.measurements
            display(eg_measurements)

        # print(f'Participant: {selected_participant.value}    Trial: {selected_trial.value}', end='\r')

# Attach the handler to the trial dropdown
selected_trial.observe(on_trial_change, names='value')

Dropdown(description='Participant:', options=('A05', 'C08', 'A02', 'C06', 'C07', 'A03', 'A04', 'C12', 'C15', '…

Dropdown(description='Trial:', options=('A05_Slow_04.csv', 'A05_Slow_03.csv', 'A05_Slow_02.csv', 'A05_Slow_01.…

Output()

In [7]:
print(selected_participant.value)
print(selected_trial.value)
eg_measurements

A05
A05_Slow_03.csv


,a_pitch,a_roll,a_yaw,omega_pitch,omega_roll,omega_yaw
0,0.125977,-0.982910,-0.168945,-45.182926,-2.987805,0.182927
1,0.125977,-0.982910,-0.168945,-45.182926,-2.987805,0.182927
2,0.125977,-0.982910,-0.168945,-45.182926,-2.987805,0.182927
3,0.125977,-0.982910,-0.168945,-45.182926,-2.987805,0.182927
4,0.115299,-0.984065,-0.167791,-45.218967,-0.104764,0.939725
...,...,...,...,...,...,...
11135,0.069336,-0.995605,-0.330078,-39.390244,-20.853659,19.756098
11136,0.069336,-0.995605,-0.330078,-39.390244,-20.853659,19.756098
11137,0.069336,-0.995605,-0.330078,-39.390244,-20.853659,19.756098
11138,0.069336,-0.995605,-0.330078,-39.390244,-20.853659,19.756098


# /

In [ ]:
from IPython.display import display
from ipywidgets import interact

# Define a function dependent on a variable
def my_function(x):
    print(f"x is now: {x}")

# Create an interactive widget
interact(my_function, x=10)


interactive(children=(IntSlider(value=10, description='x', max=30, min=-10), Output()), _dom_classes=('widget-…

<function __main__.my_function(x)>